# this should be in backend
testing here for convenience

In [6]:
import os
import json
import pandas as pd

import torch
from openai import OpenAI
from sentence_transformers import SentenceTransformer, util

import mysql.connector
from dotenv import load_dotenv, find_dotenv
from sqlalchemy import create_engine
from sqlalchemy import text as sqltext
load_dotenv(find_dotenv())

from svs import SVS
from queryOpenAI import run_engine

C:\Users\antho\AppData\Local\Temp\ipykernel_26844\1240516262.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
c:\Users\antho\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
host = os.getenv("DB_HOST")
user = os.getenv("DB_USERNAME")
password = os.getenv("DB_PASSWORD")
database = os.getenv("DB_NAME")

In [8]:
# preloading slow things
model = SentenceTransformer('distilbert-multilingual-nli-stsb-quora-ranking')
f = open('../models/mimic-iii/qqdict.json')
qqdict = json.load(f)
corpus_embedding = torch.tensor(pd.read_csv("../models/mimic-iii/corpus.csv").values).to(torch.float32)

client = OpenAI(api_key=os.getenv("OAI_KEY"))

In [6]:
conn_string = f'mysql+mysqlconnector://{user}:{password}@{host}/{database}'
engine = create_engine(conn_string, echo=True)
conn = engine.connect()

c:\Users\antho\AppData\Local\Programs\Python\Python310\lib\site-packages\mysql\connector\abstracts.py:130: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if group is 'connector_python':
c:\Users\antho\AppData\Local\Programs\Python\Python310\lib\site-packages\mysql\connector\abstracts.py:130: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if group is 'connector_python':


DatabaseError: (mysql.connector.errors.DatabaseError) 1105 (HY000): unknown error: Code: UNAVAILABLE
server does not allow insecure connections, client must use SSL/TLS

(Background on this error at: https://sqlalche.me/e/20/4xp6)

In [1]:
question = "how much does the price mean for a inject/infuse nec?"

In [9]:
def backend(question):
    qqlist = [qqdict[str(i)] for i in SVS(question, k=30, corpus=corpus_embedding, model=model)]

    relevant_qtext = "# Q: \t" + "\n# Q: \t".join(["\n# A: \t".join(rel) for rel in qqlist])
    test_prompt = open('../models/mimic-iii/codex_apidoc.txt').read()

    # pred_query = run_engine(question, test_prompt, relevant_qtext)

    # print(pred_query)
    # print()

    max_iter = 3

    result = None
    for i in range(max_iter):
        print(test_prompt)
        print()
        try:
            pred_query = run_engine(question, test_prompt, relevant_qtext)
            # result = conn.execute(sqltext(pred_query)).fetchall()

            # if len(result) == 0:
            #     test_prompt = "0 rows returned, try again.\n" + test_prompt
            #     continue

            break
        except Exception as e:
            test_prompt = str(e) + "\n" + test_prompt
            print(e)

    return pred_query
    return result
    
backend(question)

# this sql database does not support LIMIT.
#
# hadm_id: hospital admission id
# dose_val_rx: dosage size
# dose_unit_rx: unit of dose
# valueom: value units over minutes 
#
### MySQL SQL tables, with their properties:
#
# patients(row_id, subject_id, gender, dob, dod)
# admissions(row_id, subject_id, hadm_id, admittime, dischtime, admission_type, admission_location, discharge_location, insurance, language, marital_status, ethnicity, age)
# d_icd_diagnoses(row_id, icd9_code, short_title, long_title)
# d_icd_procedures(row_id, icd9_code, short_title, long_title)
# d_labitems(row_id, itemid, label)
# d_items(row_id, itemid, label, linksto)
# diagnoses_icd(row_id, subject_id, hadm_id, icd9_code, charttime)
# procedures_icd(row_id, subject_id, hadm_id, icd9_code, charttime)
# labevents(row_id, subject_id, hadm_id, itemid, charttime, valuenum, valueuom)
# prescriptions(row_id, subject_id, hadm_id, startdate, enddate, drug, dose_val_rx, dose_unit_rx, route)
# cost(row_id, subject_id, hadm_id

"SELECT DISTINCT COST.COST FROM COST WHERE COST.EVENT_TYPE = 'PROCEDURES_ICD' AND COST.EVENT_ID IN ( SELECT PROCEDURES_ICD.ROW_ID FROM PROCEDURES_ICD WHERE PROCEDURES_ICD.ICD9_CODE = ( SELECT D_ICD_PROCEDURES.ICD9_CODE FROM D_ICD_PROCEDURES WHERE D_ICD_PROCEDURES.SHORT_TITLE = 'INJECT/INFUSE NEC' ) );"

In [61]:
# backend("what is the range of patient numbers?")
# backend("how many patients do we have in the system?")
# backend("what is the admission history of patient 41976?")
# backend("what other patients have same diagnosis' as patient 41976?")
# backend("what are the four most frequently ordered procedures for patients")
backend("which patients have similar symptoms to patient 41976?")






# backend("what are the four most frequently ordered procedures for patients who were previously diagnosed with pure hypercholesterolem within 2 months, in 2101?")
# backend("has patient 594 received any lab test since 05/2104?")
# ? backend("during this month patient 15794 had been prescribed d5w?")
# ? backend("has patient 69895 been prescribed rocuronium on the first hospital visit?")


# this sql database does not support LIMIT.
#
# hadm_id: hospital admission id
# dose_val_rx: dosage size
# dose_unit_rx: unit of dose
# valueom: value units over minutes 
#
### MySQL SQL tables, with their properties:
#
# patients(row_id, subject_id, gender, dob, dod)
# admissions(row_id, subject_id, hadm_id, admittime, dischtime, admission_type, admission_location, discharge_location, insurance, language, marital_status, ethnicity, age)
# d_icd_diagnoses(row_id, icd9_code, short_title, long_title)
# d_icd_procedures(row_id, icd9_code, short_title, long_title)
# d_labitems(row_id, itemid, label)
# d_items(row_id, itemid, label, linksto)
# diagnoses_icd(row_id, subject_id, hadm_id, icd9_code, charttime)
# procedures_icd(row_id, subject_id, hadm_id, icd9_code, charttime)
# labevents(row_id, subject_id, hadm_id, itemid, charttime, valuenum, valueuom)
# prescriptions(row_id, subject_id, hadm_id, startdate, enddate, drug, dose_val_rx, dose_unit_rx, route)
# cost(row_id, subject_id, hadm_id

'SELECTPATIENTS.SUBJECT_ID\nFROM PATIENTS\nWHERE PATIENTS.SUBJECT_ID != 41976\nAND PATIENTS.GENDER = (SELECT PATIENTS.GENDER FROM PATIENTS WHERE PATIENTS.SUBJECT_ID = 41976)\nAND PATIENTS.DOB = (SELECT PATIENTS.DOB FROM PATIENTS WHERE PATIENTS.SUBJECT_ID = 41976)\nAND PATIENTS.ETHNICITY = (SELECT PATIENTS.ETHNICITY FROM PATIENTS WHERE PATIENTS.SUBJECT_ID = 41976)\nAND PATIENTS.MARITAL_STATUS = (SELECT PATIENTS.MARITAL_STATUS FROM PATIENTS WHERE PATIENTS.SUBJECT_ID = 41976);'

In [62]:
print(


'SELECTPATIENTS.SUBJECT_ID\nFROM PATIENTS\nWHERE PATIENTS.SUBJECT_ID != 41976\nAND PATIENTS.GENDER = (SELECT PATIENTS.GENDER FROM PATIENTS WHERE PATIENTS.SUBJECT_ID = 41976)\nAND PATIENTS.DOB = (SELECT PATIENTS.DOB FROM PATIENTS WHERE PATIENTS.SUBJECT_ID = 41976)\nAND PATIENTS.ETHNICITY = (SELECT PATIENTS.ETHNICITY FROM PATIENTS WHERE PATIENTS.SUBJECT_ID = 41976)\nAND PATIENTS.MARITAL_STATUS = (SELECT PATIENTS.MARITAL_STATUS FROM PATIENTS WHERE PATIENTS.SUBJECT_ID = 41976);'



)

SELECTPATIENTS.SUBJECT_ID
FROM PATIENTS
WHERE PATIENTS.SUBJECT_ID != 41976
AND PATIENTS.GENDER = (SELECT PATIENTS.GENDER FROM PATIENTS WHERE PATIENTS.SUBJECT_ID = 41976)
AND PATIENTS.DOB = (SELECT PATIENTS.DOB FROM PATIENTS WHERE PATIENTS.SUBJECT_ID = 41976)
AND PATIENTS.ETHNICITY = (SELECT PATIENTS.ETHNICITY FROM PATIENTS WHERE PATIENTS.SUBJECT_ID = 41976)
AND PATIENTS.MARITAL_STATUS = (SELECT PATIENTS.MARITAL_STATUS FROM PATIENTS WHERE PATIENTS.SUBJECT_ID = 41976);


In [8]:
test_prompt = open('../models/mimic-iii/codex_apidoc.txt').read()

In [5]:
print(question)
print()
print("SELECT COUNT(*)>0 \nFROM PRESCRIPTIONS \nWHERE PRESCRIPTIONS.HADM_ID IN \n( \n    SELECT ICUSTAYS.HADM_ID \n    FROM ICUSTAYS \n    WHERE ICUSTAYS.SUBJECT_ID = 22973 \n    ORDER BY ICUSTAYS.INTIME DESC \n    LIMIT 1 \n) \nAND PRESCRIPTIONS.DRUG = 'INSULIN';")

Was patient 22973 prescribed insulin the last time they were at the ICU?

SELECT COUNT(*)>0 
FROM PRESCRIPTIONS 
WHERE PRESCRIPTIONS.HADM_ID IN 
( 
    SELECT ICUSTAYS.HADM_ID 
    FROM ICUSTAYS 
    WHERE ICUSTAYS.SUBJECT_ID = 22973 
    ORDER BY ICUSTAYS.INTIME DESC 
    LIMIT 1 
) 
AND PRESCRIPTIONS.DRUG = 'INSULIN';


In [25]:
text = '''SELECT( SELECT CHARTEVENTS.VALUENUM \n  FROM CHARTEVENTS \n  WHERE CHARTEVENTS.ICUSTAY_ID IN \n    ( SELECT ICUSTAYS.ICUSTAY_ID \n      FROM ICUSTAYS \n      WHERE ICUSTAYS.HADM_ID IN \n        ( SELECT ADMISSIONS.HADM_ID \n          FROM ADMISSIONS \n          WHERE ADMISSIONS.SUBJECT_ID = 6215 \n          AND ADMISSIONS.DISCHTIME IS NOT NULL \n          ORDER BY ADMISSIONS.ADMITTIME ASC LIMIT 1 \n        ) \n    ) \n  AND CHARTEVENTS.ITEMID IN \n    ( SELECT D_ITEMS.ITEMID \n      FROM D_ITEMS \n      WHERE D_ITEMS.LABEL = 'ADMIT WT' \n      AND D_ITEMS.LINKSTO = 'CHARTEVENTS' \n    ) \n  ORDER BY CHARTEVENTS.CHARTTIME ASC LIMIT 1 OFFSET 1 \n) - \n( SELECT CHARTEVENTS.VALUENUM \n  FROM CHARTEVENTS \n  WHERE CHARTEVENTS.ICUSTAY_ID IN \n    ( SELECT ICUSTAYS.ICUSTAY_ID \n      FROM ICUSTAYS \n      WHERE ICUSTAYS.HADM_ID IN \n        ( SELECT ADMISSIONS.HADM_ID \n          FROM ADMISSIONS \n          WHERE ADMISSIONS.SUBJECT_ID = 6215 \n          AND ADMISSIONS.DISCHTIME IS NOT NULL \n          ORDER BY ADMISSIONS.ADMITTIME ASC LIMIT 1 \n        ) \n    ) \n  AND CHARTEVENTS.ITEMID IN \n    ( SELECT D_ITEMS.ITEMID \n      FROM D_ITEMS \n      WHERE D_ITEMS.LABEL = 'ADMIT WT' \n      AND D_ITEMS.LINKSTO = 'CHARTEVENTS' \n    ) \n  ORDER BY CHARTEVENTS.CHARTTIME ASC LIMIT 1 \n);'''

In [28]:
conn_string = f'mysql+mysqlconnector://{user}:{password}@{host}/{database}'
engine = create_engine(conn_string, echo=True)
connection = engine.connect()
test = 0
try:
    result = connection.execute(sqltext(text)).fetchall()
except Exception as e:
    print(e)
    test = e


2024-02-11 07:52:14,613 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2024-02-11 07:52:14,613 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-11 07:52:14,759 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2024-02-11 07:52:14,760 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-11 07:52:14,831 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2024-02-11 07:52:14,832 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-11 07:52:14,947 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-11 07:52:14,947 INFO sqlalchemy.engine.Engine SELECT( SELECT CHARTEVENTS.VALUENUM 
  FROM CHARTEVENTS 
  WHERE CHARTEVENTS.ICUSTAY_ID IN 
    ( SELECT ICUSTAYS.ICUSTAY_ID 
      FROM ICUSTAYS 
      WHERE ICUSTAYS.HADM_ID IN 
        ( SELECT ADMISSIONS.HADM_ID 
          FROM ADMISSIONS 
          WHERE ADMISSIONS.SUBJECT_ID = 6215 
          AND ADMISSIONS.DISCHTIME IS NOT NULL 
          ORDER BY ADMISSIONS.ADMITTIME ASC LIMIT 1 
        ) 
    ) 
  AND CHARTEVENTS.ITEMID IN 
    ( 

In [29]:
test

sqlalchemy.exc.ProgrammingError('(mysql.connector.errors.ProgrammingError) 1235 (42000): target: ehrudite-alt.-.primary: vttablet: This version of MySQL doesn\'t yet support \'LIMIT & IN/ALL/ANY/SOME subquery\' (errno 1235) (sqlstate 42000) (CallerID: xsylitx3w6v21qdtinlf): Sql: "select (select CHARTEVENTS.VALUENUM from CHARTEVENTS where CHARTEVENTS.ICUSTAY_ID in (select ICUSTAYS.ICUSTAY_ID from ICUSTAYS where ICUSTAYS.HADM_ID in (select ADMISSIONS.HADM_ID from ADMISSIONS where ADMISSIONS.SUBJECT_ID = :ADMISSIONS_SUBJECT_ID /* INT64 */ and ADMISSIONS.DISCHTIME is not null order by ADMISSIONS.ADMI')

In [35]:
t = "SELECT AVG(COST.COST) \nFROM COST \nWHERE COST.HADM_ID IN \n( \n    SELECT DIAGNOSES_ICD.HADM_ID \n    FROM DIAGNOSES_ICD \n    WHERE DIAGNOSES_ICD.ICD9_CODE = \n    ( \n        SELECT D_ICD_DIAGNOSES.ICD9_CODE \n        FROM D_ICD_DIAGNOSES \n        WHERE D_ICD_DIAGNOSES.SHORT_TITLE = 'DIABETES' \n    ) \n);".upper().replace("\n", "")
t

"SELECT AVG(COST.COST) FROM COST WHERE COST.HADM_ID IN (     SELECT DIAGNOSES_ICD.HADM_ID     FROM DIAGNOSES_ICD     WHERE DIAGNOSES_ICD.ICD9_CODE =     (         SELECT D_ICD_DIAGNOSES.ICD9_CODE         FROM D_ICD_DIAGNOSES         WHERE D_ICD_DIAGNOSES.SHORT_TITLE = 'DIABETES'     ) );"